In [9]:
pip install webdriver-manager


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.0 -> 24.1.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [8]:
pip install selenium pandas beautifulsoup4 fake_useragent colorama


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.0 -> 24.1.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [1]:
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.chrome.options import Options as ChromeOptions
from selenium.webdriver.firefox.options import Options as FirefoxOptions
import time
import re
from fake_useragent import UserAgent
import colorama
from bs4 import BeautifulSoup

# Setup
colorama.init()
RED = colorama.Fore.LIGHTRED_EX
CYAN = colorama.Fore.CYAN
GREEN = colorama.Fore.LIGHTGREEN_EX
YELLOW = colorama.Fore.YELLOW
MAGENTA = colorama.Fore.LIGHTMAGENTA_EX
BLUE = colorama.Fore.BLUE
RESET = colorama.Fore.RESET

def convertSeconds(seconds):
    seconds = float(seconds)
    hours = int(seconds / 3600)
    seconds %= 3600
    minutes = int(seconds / 60)
    seconds %= 60
    return f"[{hours}:{minutes}:{seconds:.2f}]"

def progressBar(total, current, start_time):
    current += 1
    empty = RED + '░'
    filled = GREEN + '█'
    border = BLUE + '|'
    percent = (current / total) * 100
    filled_count = int(percent)
    empty_count = 100 - filled_count
    try:
        t = time.time()
        time_left = (t - start_time) * (total / current) - (t - start_time)
    except Exception as e:
        print(f'Error in calculating time left: {e}')
        time_left = 0
    time_left = convertSeconds(time_left)
    print(f'\r{border}{filled * filled_count}{empty * empty_count}{border}{YELLOW} {percent:.2f}% [{current}/{total}] {CYAN}{time_left} {RESET}', end='')
    if percent == 100:
        time_left = convertSeconds(time.time() - start_time)
        print(f'\r{border}{filled * filled_count}{empty * empty_count}{border}{YELLOW} {percent:.2f}% [{current}/{total}] {CYAN}{time_left} {MAGENTA}\n', end=RESET)

def setup_driver(use_chrome=True):
    user = UserAgent().random
    options = ChromeOptions() if use_chrome else FirefoxOptions()
    options.add_argument("--window-size=1920,1080")
    options.add_argument("--no-sandbox")
    options.add_argument("--headless")
    options.add_argument("--disable-gpu")
    options.add_argument("--disable-crash-reporter")
    options.add_argument("--disable-extensions")
    options.add_argument("--disable-in-process-stack-traces")
    options.add_argument("--disable-logging")
    options.add_argument("--disable-dev-shm-usage")
    options.add_argument("--incognito")
    options.add_argument("--log-level=3")
    options.add_argument("--output=/dev/null")
    options.add_argument(f"user-agent={user}")
    driver = webdriver.Chrome(options=options) if use_chrome else webdriver.Firefox(options=options)
    driver.maximize_window()
    return driver, ActionChains(driver)

def get_data(element):
    html = element.get_attribute('innerHTML')
    soup = BeautifulSoup(html, 'html.parser')
    return soup.get_text("\n", strip=True) + '\n'

def click(driver, element):
    driver.execute_script("arguments[0].click();", element)

def get_price(value):
    if value:
        value = value.replace(',', '').replace(' ', '')
        value = re.search(r'\d+\.?\d*', value)
        if value:
            value = value.group()
            return eval(value)
    return ''

def scrape_courses(driver, mainUrl, file_name):
    data = {
        'id': [],
        'url': [],
        'name': [],
        'price': [],
        'course_no': [],
        'selection': [],
        'time_options': []
    }
    driver.get(mainUrl)
    time.sleep(5)
    click(driver, driver.find_elements(By.XPATH, '//a[contains(@class,"results-per-page-button")]')[-1])
    time.sleep(5)
    try:
        total_pages = get_data(driver.find_element(By.XPATH, '//div[@class="pagination"]')).strip()
        total_pages = [int(i) for i in re.findall(r'\d+', total_pages)]
        total_pages = max(total_pages)
        print(f'Total Pages: {total_pages}')
    except:
        total_pages = 5
    links = []
    page = 1
    while page <= total_pages:
        try:
            print(f'Extracting Page: {page}')
            courses_links = driver.find_elements(By.XPATH, '//div[@class="result"]/a')
            pagelinks = [element.get_attribute('href') for element in courses_links]
            links += pagelinks
            print(f'No of links on page {page}: {len(pagelinks)}')
            nxtBtn = driver.find_elements(By.XPATH, '//li[@class="arrow"]/a')[-1]
            click(driver, nxtBtn)
            time.sleep(5)
            page += 1
        except Exception as e:
            print(f'Error in getting All Links: {e}')
    links = list(set(links))
    total_links = len(links)
    print(f'Total Links: {len(links)}')
    st = time.time()
    for current_link, link in enumerate(links):
        try:
            driver.get(link)
            time.sleep(5)
            url = driver.current_url
            try:
                name = get_data(driver.find_element(By.XPATH, '//div[@class="course-title"]')).strip()
            except:
                name = ''
            try:
                price = get_price(get_data(driver.find_element(By.XPATH, '//div[@class="price--min"]')).strip())
            except:
                price = ''
            try:
                course_no = get_data(driver.find_element(By.XPATH, '//div[@class="course-number"]')).strip()
            except:
                course_no = ''
            try:
                selection = get_data(driver.find_element(By.XPATH, '//div[@class="user-selection"]')).strip()
            except:
                selection = ''
            try:
                time_options = get_data(driver.find_element(By.XPATH, '//div[@class="selectize-control single"]'))
                time_options = time_options.split('\n')
                for time_option in time_options:
                    time_option = time_option.strip()
                    if time_option:
                        data['id'].append(current_link)
                        data['url'].append(url)
                        data['name'].append(name)
                        data['price'].append(price)
                        data['course_no'].append(course_no)
                        data['selection'].append(selection)
                        data['time_options'].append(time_option)
            except:
                time_options = ''
                data['id'].append(current_link)
                data['url'].append(url)
                data['name'].append(name)
                data['price'].append(price)
                data['course_no'].append(course_no)
                data['selection'].append(selection)
                data['time_options'].append(time_options)
            progressBar(total_links, current_link, st)
        except Exception as e:
            print(f'Error in getting Course Data: {e}')
            print(f'Error in getting Course Data: {link}')
    df = pd.DataFrame(data)
    df.to_excel(file_name, index=False)
    print(f'Data Saved to {file_name}')

def process_data(file_names, output_csv):
    combined_data = []
    for file_name in file_names:
        df = pd.read_excel(file_name)
        time.sleep(5)
        
        for _, row in df.iterrows():
            course_no = re.search(r'GK\s*#\s*\d+', row['course_no'])
            if course_no:
                course_no = course_no.group()
            else:
                course_no = ''
            selection = re.search(r'\d+\s*Days', row['selection'])
            if selection:
                duration = selection.group()
            else:
                duration = ''
            time_options_parts = row['time_options'].split('|')
            if len(time_options_parts) > 1:
                start_date = time_options_parts[1].strip()
                slot_time = time_options_parts[2].strip()
            else:
                start_date = ''
                slot_time = ''
            combined_data.append({
                'id': row['id'],
                'url': row['url'],
                'coursename': row['name'],
                'price': row['price'],
                'priceunit': 'USD',
                'course_code': course_no,
                'duration': duration,
                'start_date': start_date,
                'end_date': '',
                'slot_time': slot_time,
                'brandname': 'Global Knowledge',
                'region': 'USA'
            })
    combined_df = pd.DataFrame(combined_data)
    combined_df.to_csv(output_csv, index=False)
    print(f'Combined data saved to {output_csv}')

def main():
    courses = [
        {"url": "https://www.globalknowledge.com/us-en/training/course-catalog/brands/aws/", "file_name": "AWSCourses_Global Knowledge.xlsx"},
        {"url": "https://www.globalknowledge.com/us-en/training/course-catalog/brands/comptia/", "file_name": "COMPTIACourses_Global Knowledge.xlsx"},
        {"url": "https://www.globalknowledge.com/us-en/training/course-catalog/brands/microsoft/", "file_name": "MICROSOFTCourses_Global Knowledge.xlsx"},
        # {"url": "https://www.globalknowledge.com/us-en/training/course-catalog/brands/cisco/", "file_name": "CISCOCourses_Global Knowledge.xlsx"},
        # {"url": "https://www.globalknowledge.com/us-en/training/course-catalog/brands/ibm/", "file_name": "IBMCourses_Global Knowledge.xlsx"}
    ]
    use_chrome = True
    driver, action = setup_driver(use_chrome)
    for course in courses:
        scrape_courses(driver, course["url"], course["file_name"])
    driver.quit()
    
    file_names = [course["file_name"] for course in courses]
    output_csv = "CombinedCourses_GlobalKnowledge.csv"
    process_data(file_names, output_csv)

if __name__ == "__main__":
    main()


Total Pages: 2
Extracting Page: 1
No of links on page 1: 80
Extracting Page: 2
No of links on page 2: 34
Total Links: 114
|░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░| 0.88% [1/114] [0:15:57.09] █░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░| 1.75% [2/114] [0:15:41.38] ██░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░| 2.63% [3/114] [0:15:57.93] ███░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░| 3.51% [4/114] [0:14:57.73] ████░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░| 4.39% [5/114] [0:15:5.18] █████░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░| 5.26% [6/114] [0:14:33.09] ██████░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░| 6.

TypeError: expected string or bytes-like object, got 'float'

In [3]:
import pandas as pd
import re

# Load the Excel file into a DataFrame
df = pd.read_excel("D:/360DT/Course_Scanner/new/Code/CISCOCourses_Global Knowledge.xlsx")

# Define the path for the output CSV file
output_csv = 'combined_data.csv'

combined_data = []

# Iterate over DataFrame rows
for _, row in df.iterrows():
    # Extract course number
    course_no = re.search(r'GK\s*#\s*\d+', str(row['course_no']))
    if course_no:
        course_no = course_no.group()
    else:
        course_no = ''
    
    # Extract duration
    selection = re.search(r'\d+\s*Days', str(row['selection']))
    if selection:
        duration = selection.group()
    else:
        duration = ''
    
    # Process time options
    time_options_parts = str(row['time_options']).split('|')
    if len(time_options_parts) > 2:
        start_date = time_options_parts[1].strip()
        slot_time = time_options_parts[2].strip()
    else:
        start_date = ''
        slot_time = ''
    
    # Append the processed data to the list
    combined_data.append({
        'id': row['id'],
        'url': row['url'],
        'coursename': row['name'],
        'price': row['price'],
        'priceunit': 'USD',
        'course_code': course_no,
        'duration': duration,
        'start_date': start_date,
        'end_date': '',  # Ensure to handle end_date if applicable
        'slot_time': slot_time,
        'brandname': 'Global Knowledge',
        'region': 'USA'
    })

# Convert the list of dictionaries to a DataFrame
combined_df = pd.DataFrame(combined_data)

# Save the DataFrame to a CSV file
combined_df.to_csv(output_csv, index=False)

print(f'Combined data saved to {output_csv}')


Combined data saved to combined_data.csv
